In [2]:
import time
from pop import Pilot

Car = Pilot.AutoCar()
for i in range(0,4):
    Car.setPixelDisplay(2**i,255,255,255)


ValueError: [Error] An error occured while reading I2C Devcie

In [ ]:
import math
import time
from pop import Pilot, LiDAR

In [220]:
def getBins(lidar, n_bins, angle_of_b):
    distance = 1200
    safe_distance = 650
    width_of_bin_0 = 350
    bins = [] 
    safe_bins = []
    points_in_bin = [] 
    points_in_safe_bin = []
    for angle in range(int(n_bins)):
        bins.append(0)
        safe_bins.append(0)
        points_in_bin.append(0)
        points_in_safe_bin.append(0)
    vectors = lidar.getVectors()
    for v in vectors:
        if v[0] <= 90 or v[0] >=270:
            if v[0] >= 270:
                angle_ = 360 - v[0]
            else:
                angle_ = v[0]
            rad = math.radians(angle_)
            if v[1]*math.sin(rad) <= width_of_bin_0/2 and v[1]*math.cos(rad) <= distance:
                points_in_bin[0] += 1
            if v[1]*math.sin(rad) <= width_of_bin_0/2 and v[1]*math.cos(rad) <= safe_distance:    
                points_in_safe_bin[0] += 1
                
        if v[0] <= 360 - angle_of_b/2 and v[0] >= angle_of_b/2:
            b = int((angle_of_b/2+v[0])/angle_of_b)
            if v[1] <= distance: 
                points_in_bin[b] += 1
            if v[1] <= safe_distance:
                points_in_safe_bin[b] += 1
                
    for b in range(int(n_bins)): 
        if points_in_bin[b] >= 2:
            bins[b] = 1
        if points_in_safe_bin[b] >= 2:
            safe_bins[b] = 1
        
    return bins, safe_bins

In [1]:
def compute_desired_bin( b, n_bins, angle_of_b, bins, safe_bins):
    bin_id = 0
    count = 0
    if b < 0: #. b in range (0,360) 
        b += 360
    index = int((b+angle_of_b/2) / angle_of_b)
    if index > n_bins-1:
        index = 0
    for i in range(int(n_bins/2)):
        if(bins[(index+i)%n_bins] == 0):
            bin_id = (index+i)%n_bins
            break
        if(bins[index-i] == 0):
            bin_id = index-i
            if bin_id < 0:
                bin_id = bin_id + n_bins 
            break
    if bins == [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]:
        return bin_id, False
    
    if bin_id <= n_bins//2 :
        for b in range(0, bin_id - 1):
            if safe_bins[b] == 1:
                return bin_id, False
        return bin_id, True
        
    elif bin_id > n_bins//2 :
        count += safe_bins[0]
        for b in range(bin_id + 1, n_bins):
            if safe_bins[b] == 1:
                return bin_id, False
        return bin_id, True

In [6]:
n_bins = int(12) #divisible by 4: 4, 8, 12, 16
angle_of_b = 360/n_bins
bins =      [1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1]
safe_bins = [0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1]

b = 30
bin_id, success = compute_desired_bin( b, n_bins, angle_of_b, bins, safe_bins)
angle = bin_id * angle_of_b
print(angle)
print(bin_id)
print(success)

120.0
4
True


In [ ]:
Car = Pilot.AutoCar()
lidar = LiDAR.Rplidar()
Car.setObstacleDistance(distance=0)
lidar.connect()
lidar.startMotor()

In [ ]:
Car.setSensorStatus(euler=1)

In [ ]:
Car.setCalibration()

In [ ]:
n_bins = int(12) #divisible by 4: 4, 8, 12, 16
angle_of_b = 360/n_bins
max_speed = 60
destination_angle = 45
while True:
    bins, safe_bins = getBins(lidar, n_bins, angle_of_b)
    print('     bins: ',bins)
    print('safe_bins: ',safe_bins)
    input('enter')

In [222]:
Car.stop()

In [ ]:
Car.alarm(scale=4, pitch = 8, duration = 2)

In [221]:
n_bins = int(12) #divisible by 4: 4, 8, 12, 16
angle_of_b = 360/n_bins
max_speed = 60
destination_angle = 45
while True:
    Car.setLamp(1, 1)
    bins, safe_bins = getBins(lidar, n_bins, angle_of_b)
    current_angle = Car.getEuler('yaw')
    b = destination_angle - current_angle
    bin_id, success = compute_desired_bin( b, n_bins, angle_of_b, bins, safe_bins)
    angle = bin_id * angle_of_b
    safety = 3
    if angle > 180:
        angle = angle - 360
    if success:
        if abs(angle) <= 15:
            steering = 0
        elif 15 < abs(angle) and abs(angle) <= 40:
            steering = 0.35
        elif 40 < abs(angle) and abs(angle) <= 65:
            steering = 0.7
        elif 65 < abs(angle) and abs(angle) <= 90:
            steering = 1
        else:
            steering = 1
                        
        if angle <= 0:
            steering = -steering
        
        for b in range(-n_bins//4 + 1, n_bins//4):
            if safe_bins[b] == 1:
                safety = 1
                break
        if safety > 1:
            for b in range(-n_bins//4 + 1, n_bins//4):
                if bins[b] == 1:
                    safety = 2
                    break
    else:
        steering = 0
        safety = 0  
        

            
     # send commands                
    Car.steering = steering                       
    if safety == 3:
        Car.forward(max_speed)
    elif safety == 2:   
        Car.forward(max_speed*4/5)
    elif safety == 1:   
        Car.forward(max_speed*3/5)    
    else:
        Car.setLamp(1, 1)
        Car.setSpeed(0) 
        Car.stop()
        
    time.sleep(0.1)             
    print(bins)
    print(safe_bins)
    print(f"bin_id: {bin_id}, can quay {angle}, da quay {Car.getEuler('yaw')-current_angle}")
    print(f"steering: {steering}, safety: {safety}")
    #input('enter')

count 0
[0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0]
[0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0]
bin_id: 2, can quay 60.0, da quay 0.0
steering: 0.7, safety: 3
count 0
[0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0]
[0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0]
bin_id: 2, can quay 60.0, da quay 0.0
steering: 0.7, safety: 3
count 0
[0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0]
[0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0]
bin_id: 2, can quay 60.0, da quay 0.0
steering: 0.7, safety: 3
count 0
[0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0]
[0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0]
bin_id: 2, can quay 60.0, da quay 0.0
steering: 0.7, safety: 3
count 0
[0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0]
bin_id: 2, can quay 60.0, da quay 0.0
steering: 0.7, safety: 3
count 0
[0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0]
bin_id: 2, can quay 60.0, da quay -333.187
steering: 0.7, safety: 3
count 0
[0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0]
bin_id: 1, can quay 30.0, da quay 0.0
steer

KeyboardInterrupt: 

In [ ]:
Car.steering = 0.8